# ECON 490: Locals and Globals (4)

## Prerequisites 
---
1. View the characteristics of any dataset using the command `describe`
2. Use `help` to learn best how to run commands
3. Understand the Stata command syntax using command `summarize`
4. Create loops using the commands `for`, `while`, `forvalues` and `foreach`

## Learning objectives:
---

1. Recognize the difference between dataset variable and Stata variables.
2. Recognize the difference between local and global Stata variables.
3. Use the command `local` to create temporary macros.
4. Use the command `global` to create permenant macros.
5. Know how you will use macros in your own research.


## 4.1 Stata Variables

In ECON 325 and ECON 326 you learned that "variables" are individual characteristics that can be indicated by using data. For example, if we had a dataset that included all of the countries in the world, we might have a variable that indicated each country's population. Another example, if we had a dataset that included a sample of persons in Canada, we might have have variable that indicated each person's marital status. These are dataset variables, and they can be qualitative (numeric) or quantiative (strings). 

There is a separate category of variables that are used in Stata that we call "macros". Macros work as placeholders for values that we want to store either temporarily or permenantly. Locals are macros that store data temporatily, usually while we are exectuting a do-file. Globals are macros that store data permanently, or at least as long as you have Stata open on your computer. Below you are going to learn how to use these tools in your own research. 

## 4.2 Locals 

The first use of local macros is to store results at some parts of your code. To help you understand how powerful this is, you should be aware that most Stata command have hidden results stored after they are run. Consider the following example


In [ ]:
sysuse auto,clear

summarize price


When we ran `summarize` above Stata produced output that was stored in a number of locals. You can access those stored results with the command `return list` (for regular commands) or `ereturn list` (for estimation commands, which we'll cover later on Module 12). Since `summarize`is not an estimation command, we can run the following:

In [ ]:
return list

Notice that Stata has reported that variables have been stored as `scalars`, where a scalar is simply a quantity. 

If we want Stata to tell us the mean price from the automobile dataset that was just calculated using `summarize`, we can use the following:

In [ ]:
display r(mean)

We can now store that scalar as a local, and use that local in other Stata commands, we can use the `local` command:

In [ ]:
local price_mean = r(mean)
display "The mean of price variable is `price_mean'." 

Let's conider an example of why creating this local might be useful. Imagine that you wanted create a new variable that is the price variable minus the mean of that same variable. You would do this if you wanted to demean that variable or, in other words, create a new price variable that has a mean of zero. You could the `generate` command along with the local we just created to do exactly that.

In [ ]:
replace price_demean = price - `price_mean'

Note that there is no output when you run this command. You might also need to know that if you try to run this command a second time, you will get an error because Stata doesn't want you to accidentally overwrite an exisiting variable. In order to correct this problem would need to use the command `replace` instead of the command `generate` command. Try it yourself above!

Let's take a look at the mean of our new variable using `summarize` again. 

In [ ]:
su price_demean

You can see that the mean is roughly zero just we as expected.

Locals are automatically generated whenever we use loops (as discussed in Module 3). 

Consider another common application here issuing a categorical variable that can take on 5 possible values. 

In [ ]:
su rep78

Note that if you run the command above that we use to `display` the mean of price, we will now get a different value. Try it yourself!

There are times were we might want to save all the possible categorical values in a local. When we use `levelsof` command, below, we can create a new local with a name that we chose - "levels_rep78".

In [ ]:
levelsof rep78, local(levels_rep)

We can do different things with this new list of values. For instance, we can now summarize a variable based on every distinct value of rep78 by creating a loop using `foreach` and looping through values of the new local we created. 

In [ ]:
foreach x in `levels_rep' {
su price if rep78 == `x'
}

## 4.2 Globals

Globals are will used to store lists of variable names, paths and/or directories that will need for our research project. 

Consider the following example where we create a global called "covariates" that is simply a list of three variable names.

In [ ]:
global covariates "price rep78 foreign"

We can now use that global anywhere would want to use those three variable names. When we want to indicate that we are using a global we refer to this type of macros with the dollar sign symbol `$`.

Here we `summarize` these three variables. 

In [ ]:
su ${covariates}

Try this yourself! In the emply cell below, `describe` these three variables using the macro we have just created. 

## 4.3 Wrapping Up

In this module we learned how Stata has its own set of variables that have some very useful applications. You will see these macros again through out these modules and will use them in your own research project.  

Just to demonstate how useful these can be, now we can run a very simple regression in which price is the dependent variable and the explanitory variables are rep78 and foreign. That command using our macro would be:

In [ ]:
regress ${covariates}

If we only wanted to include observations were price is above average, then using the local we created earlier in this module the regression woud be:

In [ ]:
regress ${covariates} if price > `price_mean'

You can see for yourself that Stata ran the regression only on a subset of the data.